***GENERATED CODE FOR medequipmentsuppliers PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
from pyspark.ml.feature import StringIndexer
import json
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'participationbegindate', 'transformation_label': 'Extract Date'}], 'feature': 'participationbegindate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
                                                'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'participationbegindate'}, {'feature_label': 'participationbegindate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('participationbegindate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'businessname', 'transformation_label': 'String Indexer'}], 'feature': 'businessname', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '39 PHARMACY CORPORATION', 'max': 'YONKERS EXPRESS PHARMACY INC', 'missing': '0', 'distinct': '127'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'businessname'}, {'feature_label': 'businessname', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('businessname')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'practicename', 'transformation_label': 'String Indexer'}], 'feature': 'practicename', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '86 BOBO PHARMACY INC', 'max': 'WONDERFUL RX PHARMACY INC', 'missing': '0', 'distinct': '467'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'practicename'}, {'feature_label': 'practicename', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('practicename')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'practiceaddress1', 'transformation_label': 'String Indexer'}], 'feature': 'practiceaddress1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1 S MAIN ST', 'max': 'ONE DAVIS SQ', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'practiceaddress1'}, {'feature_label': 'practiceaddress1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('practiceaddress1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'practiceaddress2', 'transformation_label': 'String Indexer'}], 'feature': 'practiceaddress2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '42', 'mean': '', 'stddev': '', 'min': '# 36', 'max': 'WALGREENS #6153', 'missing': '458', 'distinct': '29'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'practiceaddress2'}, {'feature_label': 'practiceaddress2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('practiceaddress2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'practicecity', 'transformation_label': 'String Indexer'}], 'feature': 'practicecity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ABBEVILLE', 'max': 'WOODBRIDGE', 'missing': '0', 'distinct': '397'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'practicecity'}, {'feature_label': 'practicecity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('practicecity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'practicestate', 'transformation_label': 'String Indexer'}], 'feature': 'practicestate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AK', 'max': 'WV', 'missing': '0', 'distinct': '43'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'practicestate'}, {'feature_label': 'practicestate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('practicestate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'specialitieslist', 'transformation_label': 'String Indexer'}], 'feature': 'specialitieslist', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Certified Other|Pharmacy', 'max': 'Prosthetic Personnel', 'missing': '0', 'distinct': '15'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'specialitieslist'}, {'feature_label': 'specialitieslist', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('specialitieslist')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'providertypelist', 'transformation_label': 'String Indexer'}], 'feature': 'providertypelist', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '23', 'mean': '', 'stddev': '', 'min': 'MEDICAL SUPPLY COMPANY WITH PEDORTHIC PERSONNEL', 'max': 'PHYSICIAN ASSISTANT', 'missing': '477', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'providertypelist'}, {'feature_label': 'providertypelist', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('providertypelist')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'supplieslist', 'transformation_label': 'String Indexer'}], 'feature': 'supplieslist', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '(TENS) Transcutaneous Electrical Nerve Stimulators and/or Supplies|Orthoses: Custom Fabricated|Orthoses: Prefabricated (Non-Custom Fabricated)|Orthoses: Off-The-Shelf', 'max': 'Surgical Dressings', 'missing': '0', 'distinct': '56'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'supplieslist'}, {'feature_label': 'supplieslist', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('supplieslist')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run medequipmentsuppliersHooks.ipynb
try:
	#sourcePreExecutionHook()

	medicalequipmentsuppliers = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Medical-Equipment-Suppliers1.csv', 'filename': 'Medical-Equipment-Suppliers1.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(medicalequipmentsuppliers)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run medequipmentsuppliersHooks.ipynb
try:
	#transformationPreExecutionHook()

	medequipmentsuppliersautofe = TransformationMain.run(medicalequipmentsuppliers,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "provider_id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "20494650.01", "stddev": "1133930.19", "min": "20310995", "max": "34356956", "missing": "0"}, "updatedLabel": "provider_id"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "acceptsassignement", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.92", "stddev": "0.27", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "acceptsassignement"}, {"transformationsData": [{"feature_label": "participationbegindate", "transformation_label": "Extract Date"}], "feature": "participationbegindate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "participationbegindate"}, {"transformationsData": [{"feature_label": "businessname", "transformation_label": "String Indexer"}], "feature": "businessname", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "39 PHARMACY CORPORATION", "max": "YONKERS EXPRESS PHARMACY INC", "missing": "0", "distinct": "127"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "businessname"}, {"transformationsData": [{"feature_label": "practicename", "transformation_label": "String Indexer"}], "feature": "practicename", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "86 BOBO PHARMACY INC", "max": "WONDERFUL RX PHARMACY INC", "missing": "0", "distinct": "467"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "practicename"}, {"transformationsData": [{"feature_label": "practiceaddress1", "transformation_label": "String Indexer"}], "feature": "practiceaddress1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1 S MAIN ST", "max": "ONE DAVIS SQ", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "practiceaddress1"}, {"transformationsData": [{"feature_label": "practiceaddress2", "transformation_label": "String Indexer"}], "feature": "practiceaddress2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "42", "mean": "", "stddev": "", "min": "# 36", "max": "WALGREENS #6153", "missing": "458", "distinct": "29"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "practiceaddress2"}, {"transformationsData": [{"feature_label": "practicecity", "transformation_label": "String Indexer"}], "feature": "practicecity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ABBEVILLE", "max": "WOODBRIDGE", "missing": "0", "distinct": "397"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "practicecity"}, {"transformationsData": [{"feature_label": "practicestate", "transformation_label": "String Indexer"}], "feature": "practicestate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AK", "max": "WV", "missing": "0", "distinct": "43"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "practicestate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "practicezip9code", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "424344798.64", "stddev": "358451425.83", "min": "2472", "max": "981993202", "missing": "0"}, "updatedLabel": "practicezip9code"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "telephonenumber", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "6144013752076.46", "stddev": "136516631952037.42", "min": "2014881230", "max": "3052621292106456", "missing": "0"}, "updatedLabel": "telephonenumber"}, {"transformationsData": [{"feature_label": "specialitieslist", "transformation_label": "String Indexer"}], "feature": "specialitieslist", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Certified Other|Pharmacy", "max": "Prosthetic Personnel", "missing": "0", "distinct": "15"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "specialitieslist"}, {"transformationsData": [{"feature_label": "providertypelist", "transformation_label": "String Indexer"}], "feature": "providertypelist", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "23", "mean": "", "stddev": "", "min": "MEDICAL SUPPLY COMPANY WITH PEDORTHIC PERSONNEL", "max": "PHYSICIAN ASSISTANT", "missing": "477", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "providertypelist"}, {"transformationsData": [{"feature_label": "supplieslist", "transformation_label": "String Indexer"}], "feature": "supplieslist", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "(TENS) Transcutaneous Electrical Nerve Stimulators and/or Supplies|Orthoses: Custom Fabricated|Orthoses: Prefabricated (Non-Custom Fabricated)|Orthoses: Off-The-Shelf", "max": "Surgical Dressings", "missing": "0", "distinct": "56"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "supplieslist"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "latitude", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "38.59", "stddev": "5.16", "min": "18.23407", "max": "48.57833", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "latitude"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "longitude", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-89.16", "stddev": "17.29", "min": "-157.8097", "max": "-66.03487", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "longitude"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "is_contracted_for_cba", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "is_contracted_for_cba"}]}))

	#transformationPostExecutionHook(medequipmentsuppliersautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run medequipmentsuppliersHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(medequipmentsuppliersautofe, ["provider_id", "practicezip9code", "telephonenumber", "latitude", "longitude", "is_contracted_for_cba", "participationbegindate_dayofmonth", "participationbegindate_month", "participationbegindate_year", "businessname_stringindexer", "practicename_stringindexer", "practiceaddress1_stringindexer", "practiceaddress2_stringindexer", "practicecity_stringindexer", "practicestate_stringindexer", "specialitieslist_stringindexer", "providertypelist_stringindexer", "supplieslist_stringindexer"], "acceptsassignement")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

